# Data ingest & Cleaning 

In [30]:
import pandas as pd
import numpy as np
import glob
import os

In [31]:

RAW_DATA_PATH = '../data/raw/' 
PROCESSED_DATA_PATH = '../data/processed/'
TARGET_QUARTER_DIR = 'faers_ascii_2024q4' 

In [32]:
# --- Column definitions (no changes needed) ---
DEMO_COLS = ['primaryid', 'caseid', 'caseversion', 'sex', 'age', 'occr_country', 'init_fda_dt']
DRUG_COLS = ['primaryid', 'drug_seq', 'drugname', 'role_cod']
REAC_COLS = ['primaryid', 'pt'] # 'pt' is the MedDRA Preferred Term


# Data Loading and Processing


In [33]:

def load_faers_file(filepath, columns):
    """Loads a single FAERS ASCII file, correctly handling the lack of a header."""
    df = pd.read_csv(
        filepath, delimiter='$', header=None, names=columns, usecols=columns,
        low_memory=False, on_bad_lines='warn'
    )
    if 'init_fda_dt' in df.columns:
        df.rename(columns={'init_fda_dt': 'receiptdate'}, inplace=True)
    return df

print(f"--- Processing quarter: {TARGET_QUARTER_DIR} ---")
quarter_path = os.path.join(RAW_DATA_PATH, TARGET_QUARTER_DIR, 'ascii')

demo_file = glob.glob(os.path.join(quarter_path, 'DEMO*.txt'))
drug_file = glob.glob(os.path.join(quarter_path, 'DRUG*.txt'))
reac_file = glob.glob(os.path.join(quarter_path, 'REAC*.txt'))

if not (demo_file and drug_file and reac_file):
    print(f"❌ ERROR: Could not find all required files in {quarter_path}")
else:
    df_demo = load_faers_file(demo_file[0], DEMO_COLS)
    df_drug = load_faers_file(drug_file[0], DRUG_COLS)
    df_reac = load_faers_file(reac_file[0], REAC_COLS)

    # Merge, Clean, and Filter
    merged_df = pd.merge(df_demo, df_drug, on='primaryid', how='left')
    final_df = pd.merge(merged_df, df_reac, on='primaryid', how='left')
    df_clean = final_df[final_df['role_cod'] == 'PS'].copy()
    df_clean.dropna(subset=['drugname', 'pt'], inplace=True)
    df_clean['drugname'] = df_clean['drugname'].str.upper().str.strip()
    df_clean = df_clean.sort_values('caseversion', ascending=True)
    df_clean = df_clean.drop_duplicates(subset=['primaryid', 'drugname', 'pt'], keep='last')

    # --- Create analysis-ready dataset (one row per report) ---
    df_analysis_ready = df_clean.groupby('primaryid').agg({
        'drugname': lambda x: sorted(list(set(x))),
        'pt': lambda x: sorted(list(set(x)))
    }).reset_index()
    df_analysis_ready.rename(columns={'primaryid': 'safetyreportid', 'drugname': 'drugs', 'pt': 'reactions'}, inplace=True)

    # --- Create clean demographics dataset (for deep dives) ---
    df_demo_clean = df_clean[['primaryid', 'age', 'sex', 'occr_country', 'receiptdate']].drop_duplicates(subset=['primaryid'])
    df_demo_clean.rename(columns={'primaryid': 'safetyreportid'}, inplace=True)

--- Processing quarter: faers_ascii_2024q4 ---



# Save Outputs


In [35]:

if not os.path.exists(PROCESSED_DATA_PATH):
        os.makedirs(PROCESSED_DATA_PATH)

# Save the two essential output files
analysis_output = f'faers_{TARGET_QUARTER_DIR}_analysis_ready.parquet'
demo_output = f'faers_{TARGET_QUARTER_DIR}_demographics_clean.parquet'
    
df_analysis_ready.to_parquet(os.path.join(PROCESSED_DATA_PATH, analysis_output))
df_demo_clean.to_parquet(os.path.join(PROCESSED_DATA_PATH, demo_output))

print(f"\n✅ Success! Clean data saved to:")
print(f"1. {analysis_output} (for signal detection)")
print(f"2. {demo_output} (for deep dive analysis)")


# --- Markdown Cell: Conclusion ---
# # Conclusion
#
# We have successfully transformed the raw, disparate text files into two clean, structured datasets. This foundational step ensures that all subsequent analysis is built on reliable, high-quality data. We are now ready to begin the search for safety signals.


✅ Success! Clean data saved to:
1. faers_faers_ascii_2024q4_analysis_ready.parquet (for signal detection)
2. faers_faers_ascii_2024q4_demographics_clean.parquet (for deep dive analysis)
